In [ ]:
import os
import json
import time
import config
from datetime import datetime
from log_utils import read_log_messages, normalize_template, normalize_template_v1, normalize_template_v2, save_templates, extract_last_template_from_history, extract_template_from_parser_responses, extract_event_templates, extract_last_template_from_history_loose
from ollama_utils import start_ollama_server, stop_ollama_server
from autogen import register_function
from evaluation import evaluate_and_save
from autogen.coding import LocalCommandLineCodeExecutor
from agent_utils import create_agent

In [ ]:
# --- Configuration ---
llm_config = config.LLM_CONFIG
task = config.TASK_PROMPT
LOG_DIR = config.LOG_DIR
RESULT_DIR = config.RESULT_DIR
WORK_DIR = config.WORK_DIR
os.makedirs(RESULT_DIR, exist_ok=True)
DESIGN = "MA-few"
model = llm_config["config_list"][0]["model"].replace(":", "-")
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
project_name = DESIGN.capitalize()
exp_name = f"{project_name}_{model}_{timestamp}"

input_log_file = "HDFS_200_sampled.log"
log_path = os.path.join(LOG_DIR, input_log_file)
logs = read_log_messages(log_path)

In [ ]:
# --- Create Agents ---
user_proxy = create_agent("conversable", "user_proxy_agent", llm_config, sys_prompt="A human admin.", description="A proxy for human input.")
log_parser = create_agent("assistant", "log_parser_agent", llm_config, sys_prompt=config.SYS_MSG_LOG_PARSER_FEW_SHOT, description="Analyze the log message in order to determine the corresponding template.")
comparator_refiner = create_agent("assistant", "comparator_refiner_agent", llm_config, sys_prompt=config.SYS_MSG_COMPARATOR_REFINER_FEW_SHOT, description="Compare templates and refine a final correct version.")
code_executor = create_agent("code_executor", "code_executor_agent", description="I execute the code provided.")


In [ ]:
# --- Register Drain Function ---
from drain_utils import parse_log_lines
register_function(
    parse_log_lines,
    caller=user_proxy,
    executor=code_executor,
    description="Parse the log message with Drain algorithm.",
)

In [ ]:
# --- Energy Measurement ---
from codecarbon import EmissionsTracker
proc = start_ollama_server()
time.sleep(5)
last_templates = []
tracker = EmissionsTracker(project_name=exp_name, output_dir=RESULT_DIR, save_to_file=True)
tracker.start()
try:
    for i, log_message in enumerate(logs):
        print(f"\n--- Processing log {i+1} ---")
        arguments = {
            "log_line": log_message,
            "log_format": r"<Date> <Time> <Pid> <Level> <Component>: <Content>",
            "regex": [r"blk_-?\d+", r"(\d+\.){3}\d+(:\d+)?"],
            "depth": 4,
            "similarity_threshold": 0.5,
        }
        parser_result = user_proxy.initiate_chat(
            recipient=log_parser,
            message=task + log_message,
            max_turns=1,
            summary_method="last_msg"
        )
        executor_result = user_proxy.initiate_chat(
            recipient=code_executor,
            message={
                "function_call": {
                    "name": "parse_log_lines",
                    "arguments": json.dumps(arguments)
                }
            },
            max_turns=1,
            summary_method="last_msg"
        )
        parser_template = parser_result.summary.strip()
        executor_template = executor_result.summary.strip()
        combined_message = f"""
            Compare and refine the following two templates generated for this log:

            Log Message:
            {log_message}

            Template from Parser Agent:
            {parser_template}

            Template from Code Executor Agent:
            {executor_template}

            Return the best refined version.
        """
        refine_result = user_proxy.initiate_chat(
            recipient=comparator_refiner,
            message=combined_message,
            max_turns=1,
            summary_method="last_msg"
        )
        last_template = extract_last_template_from_history(refine_result.chat_history, "comparator_refiner_agent")
        if last_template:
            last_templates.append(last_template)
        else:
            last_templates.append("NONE")
            print(f"No valid template extracted from log_parser_agent for log {i}")
finally:
    emissions = tracker.stop()
    print(f"\n🌱 Emissions: {emissions} kg CO2")
    stop_ollama_server(proc)

In [ ]:
# --- Save Templates ---
save_templates(last_templates, llm_config, DESIGN, RESULT_DIR)

# --- Evaluation ---
ground_truth_file = os.path.join(LOG_DIR, "HDFS_200_sampled_log_structured.csv")
results = evaluate_and_save(normalize_template, last_templates, ground_truth_file, DESIGN, RESULT_DIR)
results_v1 = evaluate_and_save(normalize_template_v1, last_templates, ground_truth_file, DESIGN, RESULT_DIR)
results_v2 = evaluate_and_save(normalize_template_v2, last_templates, ground_truth_file, DESIGN, RESULT_DIR)
